<a href="https://colab.research.google.com/github/omid-sar/Interview-Preparation/blob/main/ML_interview/ML_int_twitter_toxic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/omid-sar/Project_Data_Hub/main/twitter_toxic_final_balanced_dataset.csv


--2024-05-07 21:27:12--  https://raw.githubusercontent.com/omid-sar/Project_Data_Hub/main/twitter_toxic_final_balanced_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5518854 (5.3M) [text/plain]
Saving to: ‘twitter_toxic_final_balanced_dataset.csv’

twitter_toxic_final 100%[===================>]   5.26M  --.-KB/s    in 0.08s   

2024-05-07 21:27:13 (63.5 MB/s) - ‘twitter_toxic_final_balanced_dataset.csv’ saved [5518854/5518854]



In [5]:
import pandas as pd
df_org = pd.read_csv("twitter_toxic_final_balanced_dataset.csv")

In [4]:
df_org

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation
...,...,...,...
56740,56740,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
56741,56741,1,"you've gone and broke the wrong heart baby, an..."
56742,56742,1,young buck wanna eat!!.. dat nigguh like I ain...
56743,56743,1,youu got wild bitches tellin you lies
